In [78]:
%load_ext watermark
# %load_ext rpy2.ipython
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
# import rpy2
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import beta
from scipy.stats import multinomial


import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from myst_nb import glue
from slugify import slugify

from typing import List



# from rpy2.robjects.packages import importr

import plastockconf as psc
import plastock as pstk

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc

pd.set_option('display.max_rows', 100)


The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [109]:
mone = pd.read_csv('data/end_pipe/macro_data_msquared.csv')
mone["date"] = pd.to_datetime(mone['date'], format='mixed', dayfirst=True)
mone["date"] = mone["date"].dt.strftime('%Y-%m-%d')
mone["échantillon"] = list(zip(mone.Plage, mone["date"]))
mtwo = pd.read_csv('data/end_pipe/macro_current.csv')
mtwo["Date"] = pd.to_datetime(mtwo.Date, format='mixed', dayfirst=True)
mtwo["Date"] = mtwo["Date"].dt.strftime('%Y-%m-%d')


new_names = {'Substrat':'substrat', 'Date':'date', 'Code':'code', 'Aire':'area', 'Quantité':'quantité'}
columns = ['échantillon', 'Plage', 'date', 'area', 'quantité', 'code']

t1 = mone[~mone.isna()][columns]

In [110]:
mone

,échantillon,Plage,substrat,date,area,slug,code,quantité,pcs/m²,region
0,"(Amphion, 2022-02-01)",Amphion,4,2022-02-01,440,amphion,G159,0,0.000000,Grand lac
1,"(Amphion, 2022-02-01)",Amphion,4,2022-02-01,440,amphion,G10,0,0.000000,Grand lac
2,"(Amphion, 2022-02-01)",Amphion,4,2022-02-01,440,amphion,G103,0,0.000000,Grand lac
3,"(Amphion, 2022-02-01)",Amphion,4,2022-02-01,440,amphion,G106,0,0.000000,Grand lac
4,"(Amphion, 2022-02-01)",Amphion,4,2022-02-01,440,amphion,G117,0,0.000000,Grand lac
...,...,...,...,...,...,...,...,...,...,...
7149,"(Vidy, 2022-10-18)",Vidy,1,2022-10-18,504,vidy,G95,17,0.033730,Grand lac
7150,"(Vidy, 2022-10-18)",Vidy,1,2022-10-18,504,vidy,G97,1,0.001984,Grand lac
7151,"(Vidy, 2022-10-18)",Vidy,1,2022-10-18,504,vidy,Gcaps,13,0.025794,Grand lac
7152,"(Vidy, 2022-10-18)",Vidy,1,2022-10-18,504,vidy,Gfoams,1,0.001984,Grand lac


In [80]:
mtwo['échantillon'] = list(zip(mtwo.Plage, mtwo.Date))
mtwo.rename(columns=new_names, inplace=True)
t2 = mtwo[columns]

In [81]:
t2.head()

,échantillon,Plage,date,area,quantité,code
0,"(Amphion, 2022-02-01)",Amphion,2022-02-01,98,2,G24
1,"(Amphion, 2022-02-01)",Amphion,2022-02-01,342,42,G24
2,"(Amphion, 2022-05-03)",Amphion,2022-05-03,342,5,G24
3,"(Amphion, 2022-07-19)",Amphion,2022-07-19,98,5,G24
4,"(Amphion, 2022-07-19)",Amphion,2022-07-19,342,11,G24


In [82]:
t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   échantillon  2290 non-null   object
 1   Plage        2290 non-null   object
 2   date         2290 non-null   object
 3   area         2290 non-null   int64 
 4   quantité     2290 non-null   int64 
 5   code         2290 non-null   object
dtypes: int64(2), object(4)
memory usage: 107.5+ KB


In [83]:
t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7154 entries, 0 to 7153
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   échantillon  7154 non-null   object
 1   Plage        7154 non-null   object
 2   date         7154 non-null   object
 3   area         7154 non-null   int64 
 4   quantité     7154 non-null   int64 
 5   code         7154 non-null   object
dtypes: int64(2), object(4)
memory usage: 335.5+ KB


In [84]:
t1['quantité'].sum()

27493

In [85]:
t2['quantité'].sum()

28483

In [86]:
t2["échantillon"].nunique()

98

In [87]:
t1["échantillon"].nunique()

98

## Unknown codes

In [88]:
mone_codes = mone.code.unique()
mtwo_codes = mtwo.code.unique()
gfrag_gfoam_gcaps = ['G21', 'G23', 'G24', 'G78', 'G79', 'G80', 'G81', 'G82',]
diff_mtwo_mone = set(mtwo_codes).difference(set(mone_codes))

codes_in_mtwo_not_in_mone = [x for x in diff_mtwo_mone if x not in gfrag_gfoam_gcaps]
codes_in_mtwo_not_in_mone

['G9492',
 'G9135',
 'G9146',
 'G9493',
 'G9147',
 'G9145',
 'G9331',
 'G9134',
 'G9491',
 'G989']

In [89]:
# codes in mtwo not in mone:
mt_wtf = mtwo[mtwo.code.isin(codes_in_mtwo_not_in_mone)]
mt_wtf['quantité'].sum()

376

## Find the errors

C'est possible que il'n'y a pas de distinction entre les deux zones pour revernges le 13 août. 

In [112]:
t2 = t2[~t2.code.isin(codes_in_mtwo_not_in_mone)]

t1_dt = t1.groupby("échantillon")[["quantité"]].sum()
t2_dt = t2.groupby("échantillon")[["quantité"]].sum()

t1_dt['s'] = [str(x) for x in t1_dt.index]
t2_dt['s'] = [str(x) for x in t2_dt.index]

t1_dt.set_index('s', inplace=True)
t2_dt.set_index('s', inplace=True)

### difference

In [116]:
print(f'Il y a {t2_dt["quantité"].sum() - t1_dt["quantité"].sum()} plus d\'objets en t2')

Il y a 614 plus d'objets en t2


In [117]:
t1_dt['t2_q'] = t1_dt.index.map(lambda x: t2_dt.loc[x, 'quantité'])
t1_dt['diff'] = t1_dt['quantité'] - t1_dt.t2_q

t1_dt['diff'].values

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, -614,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

### l'échantillon

In [100]:
t1_dt[t1_dt['diff'] == -614]

,quantité,t2_q,diff
s,,,
"('Préverenges', '2022-08-13')",36,650,-614


In [101]:
t1samp = t1.loc[(t1.Plage=="Préverenges") & (t1["date"] == "2022-08-13")]
t2samp = t2.loc[(t2.Plage=="Préverenges") & (t2["date"] =="2022-08-13")]

In [120]:
mone[(mone.Plage == "Préverenges")&(mone['date'] == "2022-08-13")]

,échantillon,Plage,substrat,date,area,slug,code,quantité,pcs/m²,region
4122,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G114,0,0.000000,Grand lac
4123,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G131,0,0.000000,Grand lac
4124,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G148,0,0.000000,Grand lac
4125,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G211,0,0.000000,Grand lac
4126,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G31,0,0.000000,Grand lac
4127,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G35,0,0.000000,Grand lac
4128,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G66,0,0.000000,Grand lac
4129,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G70,0,0.000000,Grand lac
4130,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G905,0,0.000000,Grand lac
4131,"(Préverenges, 2022-08-13)",Préverenges,1,2022-08-13,150,preverenges,G914,0,0.000000,Grand lac


In [119]:
mtwo[(mtwo.Plage == "Préverenges")&(mtwo['Date'] == "2022-08-13")]

,Plage,Aire,Temps,Position,Substrat,Date,Saison,Objet,Détail,Code,OPSAR,UNEP,Quantité,Poids
63,Préverenges,961,490,2,NaN,2022-08-13,3,Ballon de baudruche,NaN,G125,49.0,RB01,2,0.001600
127,Préverenges,961,490,2,NaN,2022-08-13,3,Bande élastique,NaN,G131,NaN,NaN,1,0.000280
186,Préverenges,961,490,2,NaN,2022-08-13,3,Bâtonnet de sucette en plastique,NaN,G31,19.0,NaN,2,0.000360
262,Préverenges,961,490,2,NaN,2022-08-13,3,Bouchon/capuchon PET,NaN,G21,15.0,PL01,3,0.005250
348,Préverenges,961,490,2,NaN,2022-08-13,3,Briquet,NaN,G26,16.0,PL10,1,0.012000
539,Préverenges,961,490,2,NaN,2022-08-13,3,Coton-tige,NaN,G95,98.0,OT02,3,0.000540
607,Préverenges,150,80,1,1.0,2022-08-13,3,Couverts,NaN,G34,6.0,PL06,1,0.008000
753,Préverenges,150,80,1,1.0,2022-08-13,3,Emballage (film) à nourriture/alimentaire/en p...,NaN,G30,19.0,NaN,7,0.004100
754,Préverenges,961,490,2,NaN,2022-08-13,3,Emballage (film) à nourriture/alimentaire/en p...,NaN,G30,19.0,NaN,21,0.012200
882,Préverenges,961,490,2,NaN,2022-08-13,3,Ficelle de moins de 1cm de diamètre,NaN,G50,32.0,PL19,19,0.076000
